# Manually Checked Stats

Naturally we need guaranteed true accent and barys responsion stats for at least one antistrophic and one polystrophic song to know that the stats code works.

I chose ach01 and ach05.

## Accentual Responsion

### Antistrophic: ach01

- ach01: 
  - Accentual responsion (confirms visualization.py): 
    - Acute:      
      - 0 + 3 + 0 + 2 + 0 + 1 + 0 + 1 + 2 = 9
    - Grave:      
      - 1 + 0 + 0 + 1 + 0 + 0 + 0 + 0 + 0 = 2
    - Circumflex: 
      - 1 + 0 + 0 + 0 + 0 + 1 + 1 + 0 + 1 = 4

My manual check confirms the accent map of `accentually_responding_syllables_of_strophes_polystrophic`.

Using `count_all_accents_canticum(tree, canticum)` we can get the total number of acutes in both strophes as 62, which I have manually confirmed by reckoning in the file `sanity_check/ach01.tsv`. Before we compute the fraction we have two count every responding syllable twice, because it actually involves two accents. Finally, this means we have a confirmed statistic of **9*2/62 ≈ 0.29**.


In [ ]:
accent_maps = [
    [
        {('205', 7): 'πάν', ('220', 7): 'τεί'}, 
        {('205', 10): 'πό', ('220', 10): 'κέ'}, 
        {('205', 13): 'ἄξ', ('220', 13): 'ρύ'}, 
        {('207', 4): "δ' ὅ", ('222', 4): 'γέ'}, 
        {('207', 14): 'φέ', ('222', 14): 'νέ'}, 
        {('210-211', 3): 'ς. Οἴ', ('225', 3): 'οί'}, 
        {('213-214', 5): 'τί', ('227', 5): 'ρί'}, 
        {('215-217', 3): 'λού', ('230-233', 3): 'νή'}, 
        {('215-217', 23): 'τό', ('230-233', 23): 'ρό'}
        ], 
    [
        {('204', 10): 'τὸ', ('219', 10): 'μὸ'}, 
        {('207', 11): 'τὰς', ('222', 11): 'γὼ'}
        ],
    [
        {('204', 1): 'Τῇ', ('219', 1): 'Νῦν'}, 
        {('210-211', 7): 'τῶν', ('225', 7): 'θροῖ'}, 
        {('211-212', 4): 'μῆς', ('226', 4): 'μοῦ'}, 
        {('215-217', 10): 'ὧ', ('230-233', 10): 'τοῖ'}
        ]
    ]

The easiest way to manually count is to print the sylls from all responding lines aligned in tabs: 

In [ ]:
from lxml import etree as ET

def extract_responsion_syllables_vertical(filename: str, responsion_value: str) -> str:
    tree = ET.parse(filename)
    root = tree.getroot()

    # Collect all <strophe> and <antistrophe> with matching responsion
    matched_blocks = []
    for tag in ('strophe', 'antistrophe'):
        matched_blocks.extend(root.xpath(f".//{tag}[@responsion='{responsion_value}']"))

    # Extract <l> elements from each matched block
    all_l_lists = [block.findall("l") for block in matched_blocks]
    max_lines = max(len(l_list) for l_list in all_l_lists)

    output_lines = []

    for line_index in range(max_lines):
        for l_list in all_l_lists:
            if line_index < len(l_list):
                l_elem = l_list[line_index]

                # Get all <syll> elements (including those in <conjecture>)
                syll_elems = l_elem.xpath(".//syll")

                merged_sylls = []
                i = 0
                while i < len(syll_elems):
                    syll = syll_elems[i]
                    text = (syll.text or "").strip()
                    if i + 1 < len(syll_elems):
                        next_syll = syll_elems[i + 1]
                        if (syll.get("resolution") == "True" and
                            next_syll.get("resolution") == "True"):
                            merged = text + (next_syll.text or "").strip()
                            merged_sylls.append(merged)
                            i += 2
                            continue
                    merged_sylls.append(text)
                    i += 1

                output_lines.append("\t".join(merged_sylls))
            else:
                output_lines.append("")  # No line at this index in this block
        output_lines.append("")  # Blank line between line groups

    return "\n".join(output_lines)

canticum = "ach05"
infix = canticum[:-2]

input_file = f"data/compiled/responsion_{infix}_compiled.xml"

output_lines = extract_responsion_syllables_vertical(input_file, canticum)

print(output_lines)
with open(f"sanity_check/{canticum}.tsv", "w") as f:
    f.write(output_lines)

### Polystrophic: ach05

`accentually_responding_syllables_of_strophes_polystrophic` yields an empty accent_map for the quartet `ach05`. We can convince ourselves that this is not simply a lack of support for polystrophy by creating a version with an artificial responsion in the first syllable:

In [ ]:
from lxml import etree
from src.stats import accentually_responding_syllables_of_strophes_polystrophic

canticum = "ach05"
infix = canticum[:-2]
xml_file = f"data/compiled/responsion_{infix}_compiled.xml"
xml_file = f"data/compiled/test/responsion_mockup_compiled.xml"

tree = etree.parse(xml_file)
strophes = tree.xpath(f'//*[self::strophe or self::antistrophe][@responsion="{canticum}"]')
accent_maps = accentually_responding_syllables_of_strophes_polystrophic(*strophes)

print("accent_maps: ", accent_maps)

Let's try a few other polystrophic songs: eq07 (4), pax01 (3), lys08 (4), ra04 (3), ra08 (4):

ach05: {'acute': 0, 'grave': 0, 'circumflex': 0}
eq07: {'acute': 0, 'grave': 0, 'circumflex': 0}
pax01: {'acute': 6, 'grave': 3, 'circumflex': 0}
lys08: {'acute': 4, 'grave': 0, 'circumflex': 0}
ra04: {'acute': 9, 'grave': 0, 'circumflex': 0}
ra08: {'acute': 8, 'grave': 0, 'circumflex': 0}

In [ ]:
from lxml import etree
from src.stats import accentually_responding_syllables_of_strophes_polystrophic, count_all_accents_canticum
from src.utils.utils import polystrophic_cantica

polystrophic_cantica.insert(0, "ach01")

for canticum in polystrophic_cantica:
    infix = canticum[:-2]
    xml_file = f"data/compiled/responsion_{infix}_compiled.xml"
    tree = etree.parse(xml_file)

    strophes = tree.xpath(f'//*[self::strophe or self::antistrophe][@responsion="{canticum}"]')
    accent_maps = accentually_responding_syllables_of_strophes_polystrophic(*strophes)

    total_accent_sums = count_all_accents_canticum(tree, canticum)
    accent_responsion_counts = {
        'acute': sum(len(d) for d in accent_maps[0]),
        'grave': sum(len(d) for d in accent_maps[1]),
        'circumflex': sum(len(d) for d in accent_maps[2])
    }

    acute_stat = accent_responsion_counts['acute'] / total_accent_sums['acute'] if total_accent_sums['acute'] > 0 else 0
    grave_stat = accent_responsion_counts['grave'] / total_accent_sums['grave'] if total_accent_sums['grave'] > 0 else 0
    circumflex_stat = accent_responsion_counts['circumflex'] / total_accent_sums['circumflex'] if total_accent_sums['circumflex'] > 0 else 0

    print(f"Statistics for {canticum}:")
    print(f"  Acute: {accent_responsion_counts['acute']} / {total_accent_sums['acute']} = {acute_stat:.2f}")
    print(f"  Grave: {accent_responsion_counts['grave']} / {total_accent_sums['grave']} = {grave_stat:.2f}")
    print(f"  Circumflex: {accent_responsion_counts['circumflex']} / {total_accent_sums['circumflex']} = {circumflex_stat:.2f}")
    print()



Polystrophic cantica:  ['ach01', 'ach05', 'eq07', 'pax01', 'lys08', 'ra04', 'ra08']
Statistics for ach01:
  Acute: 18 / 62 = 0.29
  Grave: 4 / 27 = 0.15
  Circumflex: 8 / 32 = 0.25

Statistics for ach05:
  Acute: 0 / 28 = 0.00
  Grave: 0 / 12 = 0.00
  Circumflex: 0 / 16 = 0.00

Statistics for eq07:
  Acute: 0 / 76 = 0.00
  Grave: 0 / 18 = 0.00
  Circumflex: 0 / 26 = 0.00

Statistics for pax01:
  Acute: 6 / 79 = 0.08
  Grave: 3 / 39 = 0.08
  Circumflex: 0 / 23 = 0.00

Statistics for lys08:
  Acute: 4 / 104 = 0.04
  Grave: 0 / 57 = 0.00
  Circumflex: 0 / 51 = 0.00

Statistics for ra04:
  Acute: 9 / 28 = 0.32
  Grave: 0 / 18 = 0.00
  Circumflex: 0 / 5 = 0.00

Statistics for ra08:
  Acute: 8 / 47 = 0.17
  Grave: 0 / 11 = 0.00
  Circumflex: 0 / 10 = 0.00



## Barys Responsion